In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

# LLM Chain

In [3]:
# We can simply question LLM if we want
# llm.invoke("how can langsmith help with testing?")

# Or we can guide LLM response with a prompt template as follows:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

# combine these into a simple LLM chain
chain = prompt | llm

chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith is a powerful tool that can greatly assist with testing by providing automated language verification and correction capabilities. Here are some ways Langsmith can help with testing:\n\n1. **Automated Language Verification**: Langsmith can automatically check for grammar, spelling, and punctuation errors in your test scripts, ensuring that they are free from language-related issues that could impact the accuracy and reliability of your tests.\n\n2. **Consistent Language Usage**: Langsmith can help enforce consistent language usage across your test scripts, making it easier to maintain and understand the tests over time.\n\n3. **Localization Testing**: Langsmith can assist with localization testing by verifying the accuracy and consistency of translated text in your test scripts.\n\n4. **Accessibility Testing**: Langsmith can help ensure that your test scripts are accessible by checking for readability and clarity issues that could impact users with disabilit

In [4]:
# Out put parse to convert message:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?" })

"Langsmith can help with testing by providing automated testing capabilities for natural language processing (NLP) models and applications. It offers tools and frameworks that enable developers and quality assurance teams to efficiently test the performance and accuracy of NLP models across various languages and use cases.\n\nSome ways Langsmith can assist with testing include:\n\n1. **Test Automation:** Langsmith provides automated testing capabilities that can run a series of predefined tests on NLP models to check for accuracy, performance, and reliability. This helps in identifying issues early in the development cycle and ensures that the models meet the desired quality standards.\n\n2. **Regression Testing:** Langsmith enables developers to easily conduct regression testing to ensure that any changes or updates to the NLP models do not negatively impact their performance or accuracy. This helps in maintaining the overall quality and reliability of the models.\n\n3. **Language Sup

In [5]:
# Retrieval Chain
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# OpenAI embedding model

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [7]:
# Build indexed data into vectorstore

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [9]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


LangSmith can help with testing by allowing developers to create datasets, run tests on their LLM applications, upload test cases in bulk, create test cases on the fly, export test cases from application traces, and run custom evaluations to score test results. Additionally, LangSmith provides a comparison view to track and diagnose regressions in test scores across multiple revisions of an application, a playground environment for rapid iteration and experimentation, and the ability to add runs as examples to datasets to expand test coverage on real-world scenarios.


# Conversation Retrieval Chain

In [11]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [12]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. 

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [14]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documen

# Agent

We will give the agent access to two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith
2. A search tool. This will let it easily answer questions that require up-to-date information.

In [15]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [16]:
# Tavily API key and setup missing
search = ""

In [17]:
tools = [retriever_tool, search]


In [18]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

ImportError: Could not import langchainhub, please install with `pip install langchainhub`.

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
agent_executor.invoke({"input": "what is the weather in SF?"})


In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

# LangServe